In [1]:
import pandas as pd
from scipy.optimize import linprog
import pulp
from pulp import *

# Import Fantasy Data

In [2]:
player_info = pd.read_csv("player_historical_info.csv")
df_day1 = pd.read_csv("sixGamePlayoffwindow.csv")

In [3]:
df_day1.insert(1, 'PLAYER_NAME', value=df_day1['First Name'].str.cat(df_day1['Last Name'], sep=' '))
df_day1.head()

,ID,PLAYER_NAME,First Name,Last Name,ID + Name,Position,Team,Opponent,Game,Time,Salary,FPPG,Injury Status,Starting
0,nba.p.4387$nba.g.13467906,Derrick Rose,Derrick,Rose,nba.p.4387$nba.g.13467906 - Derrick Rose,PG,NY,MIA,NY@MIA,3:30PM EDT,10,10.2,,
1,nba.p.5054$nba.g.13467906,Evan Fournier,Evan,Fournier,nba.p.5054$nba.g.13467906 - Evan Fournier,SF,NY,MIA,NY@MIA,3:30PM EDT,10,11.6,,
2,nba.p.5318$nba.g.13467906,Julius Randle,Julius,Randle,nba.p.5318$nba.g.13467906 - Julius Randle,PF,NY,MIA,NY@MIA,3:30PM EDT,30,42.2,,
3,nba.p.5843$nba.g.13467906,Josh Hart,Josh,Hart,nba.p.5843$nba.g.13467906 - Josh Hart,SF,NY,MIA,NY@MIA,3:30PM EDT,20,27.7,,
4,nba.p.5856$nba.g.13467906,Isaiah Hartenstein,Isaiah,Hartenstein,nba.p.5856$nba.g.13467906 - Isaiah Hartenstein,C,NY,MIA,NY@MIA,3:30PM EDT,10,18.1,,


In [4]:
df_day1 = pd.merge(df_day1, player_info, on='PLAYER_NAME')


In [5]:
df_day1['Ratio'] = df_day1['Y_pred']/df_day1['pred_sigma']

In [6]:
df_day1.head()

,ID,PLAYER_NAME,First Name,Last Name,ID + Name,Position,Team,Opponent,Game,Time,Salary,FPPG,Injury Status,Starting,pred_sigma,Y_pred,Ratio
0,nba.p.4387$nba.g.13467906,Derrick Rose,Derrick,Rose,nba.p.4387$nba.g.13467906 - Derrick Rose,PG,NY,MIA,NY@MIA,3:30PM EDT,10,10.2,,,11.002471,10.467307,0.951360
1,nba.p.5054$nba.g.13467906,Evan Fournier,Evan,Fournier,nba.p.5054$nba.g.13467906 - Evan Fournier,SF,NY,MIA,NY@MIA,3:30PM EDT,10,11.6,,,9.550019,0.000000,0.000000
2,nba.p.5318$nba.g.13467906,Julius Randle,Julius,Randle,nba.p.5318$nba.g.13467906 - Julius Randle,PF,NY,MIA,NY@MIA,3:30PM EDT,30,42.2,,,10.731120,29.967511,2.792580
3,nba.p.5843$nba.g.13467906,Josh Hart,Josh,Hart,nba.p.5843$nba.g.13467906 - Josh Hart,SF,NY,MIA,NY@MIA,3:30PM EDT,20,27.7,,,10.220513,26.263104,2.569646
4,nba.p.5856$nba.g.13467906,Isaiah Hartenstein,Isaiah,Hartenstein,nba.p.5856$nba.g.13467906 - Isaiah Hartenstein,C,NY,MIA,NY@MIA,3:30PM EDT,10,18.1,,,10.099764,20.009524,1.981187


# Model 1

In [7]:
# Parameters
S = df_day1["Salary"]
P = df_day1["Position"]
V = df_day1["FPPG"]

#Model
model = LpProblem("FantasyBasketball", LpMaximize)

#Decision Variable
player_vars = LpVariable.dicts("player", df_day1.index, 0, 1, LpBinary)

#Objective
model += lpSum(V[i] * player_vars[i] for i in df_day1.index)

#Constraints
model += sum(S[i] * player_vars[i] for i in df_day1.index) <= 200 # Remain under $200
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Injury Status"] == "INJ") == 0
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Injury Status"] == "O") == 0
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Injury Status"] == "GTD") == 0

##Number of Plyers assigned to position in the line-up
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PG") >= 1
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PG") <= 3
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PG") + sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SG") <= 4
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SG") >= 1
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SG") <= 3
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SF") >= 1
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SF") <= 3
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SF") + sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PF") <= 4
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PF") >= 1
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PF") <= 3
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PG") + sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SG") + sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "C") <=5
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SF") + sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PF") + sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "C") <=5
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "C") >= 1
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "C") <= 2

##Number of players in the lineup
model += sum(player_vars[i] for i in df_day1.index) == 8 # Number of pLayers in the line-up

#Solver
status = model.solve()

#Line-up dataframe
if LpStatus[model.status]=="Optimal":
    print("Optimal Line Up:")
    data = []
    for i in df_day1.index:
        if player_vars[i].value()==1:
            data.append([df_day1.loc[i,"ID + Name"], df_day1.loc[i,"Position"], df_day1.loc[i,"Salary"], df_day1.loc[i,"Injury Status"],df_day1.loc[i,"FPPG"]])
            print(df_day1.loc[i,"ID + Name"],"|", df_day1.loc[i,"Position"], "|", df_day1.loc[i,"Salary"], "|", df_day1.loc[i,"Injury Status"], "|",df_day1.loc[i,"FPPG"] )
    
    df_lineup1 = pd.DataFrame(data, columns=["ID + Name", "Position", "Salary", "Injury Status", "FPPG"])
    print(df_lineup1["FPPG"].sum())
else:
    print("No Optimal Line Up Found")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.7/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/2f411943077e436e864debfd79298b42-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/2f411943077e436e864debfd79298b42-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 23 COLUMNS
At line 508 RHS
At line 527 BOUNDS
At line 581 ENDATA
Problem MODEL has 18 rows, 53 columns and 326 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 278.833 - 0.00 seconds
Cgl0002I 8 variables fixed
Cgl0004I processed model has 11 rows, 45 columns (45 integer (45 of which binary)) and 225 elements
Cutoff increment increased from 1e-05 to 0.0999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.666667
Cbc0038I Pass   1: suminf.    0.18182 (2) obj. -277.382 iterations 4
Cbc0038I Solution found of -244.2
Cbc0038I Before m

In [8]:
# Parameters
S = df_day1["Salary"]
P = df_day1["Position"]
V = df_day1["Y_pred"]

#Model
model = LpProblem("FantasyBasketball", LpMaximize)

#Decision Variable
player_vars = LpVariable.dicts("player", df_day1.index, 0, 1, LpBinary)

#Objective
model += lpSum(V[i] * player_vars[i] for i in df_day1.index)

#Constraints
model += sum(S[i] * player_vars[i] for i in df_day1.index) <= 200 # Remain under $200
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Injury Status"] == "INJ") == 0
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Injury Status"] == "O") == 0
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Injury Status"] == "GTD") == 0

##Number of Plyers assigned to position in the line-up
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PG") >= 1
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PG") <= 3
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PG") + sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SG") <= 4
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SG") >= 1
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SG") <= 3
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SF") >= 1
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SF") <= 3
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SF") + sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PF") <= 4
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PF") >= 1
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PF") <= 3
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PG") + sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SG") + sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "C") <=5
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "SF") + sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "PF") + sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "C") <=5
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "C") >= 1
model += sum(player_vars[i] for i in df_day1.index if df_day1.loc[i, "Position"] == "C") <= 2

##Number of players in the lineup
model += sum(player_vars[i] for i in df_day1.index) == 8 # Number of pLayers in the line-up

#Solver
status = model.solve()

#Line-up dataframe
if LpStatus[model.status]=="Optimal":
    print("Optimal Line Up:")
    data = []
    for i in df_day1.index:
        if player_vars[i].value()==1:
            data.append([df_day1.loc[i,"ID + Name"], df_day1.loc[i,"Position"], df_day1.loc[i,"Salary"], df_day1.loc[i,"Injury Status"],df_day1.loc[i,"FPPG"]])
            print(df_day1.loc[i,"ID + Name"],"|", df_day1.loc[i,"Position"], "|", df_day1.loc[i,"Salary"], "|", df_day1.loc[i,"Injury Status"], "|",df_day1.loc[i,"FPPG"] )
    
    df_lineup1 = pd.DataFrame(data, columns=["ID + Name", "Position", "Salary", "Injury Status", "FPPG"])
    print(df_lineup1["FPPG"].sum())
else:
    print("No Optimal Line Up Found")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.7/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b6b7a22b41c54c708b8c6d58c3f0822c-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/b6b7a22b41c54c708b8c6d58c3f0822c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 23 COLUMNS
At line 506 RHS
At line 525 BOUNDS
At line 579 ENDATA
Problem MODEL has 18 rows, 53 columns and 326 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 262.89 - 0.00 seconds
Cgl0002I 8 variables fixed
Cgl0004I processed model has 11 rows, 45 columns (45 integer (45 of which binary)) and 225 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.923077
Cbc0038I Solution found of -260.579
Cbc0038I Before mini branch and bound, 43 integers at bound fixed and 0 continuous
Cbc0038I Full problem 11 rows 45 columns, reduced 